In [ ]:
from features import *
import librosa
import librosa.display as display
import scipy
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from model_02_functions import *
import sklearn
from train_period_checker import max_pooling
import matplotlib.colors as colors

In [ ]:
audio_data, samplerate = librosa.load('train_recordings/hour0001.wav', sr=3200)
no_train_audio, samplerate = librosa.load('train_recordings/hour0301.wav',sr=3200)

In [ ]:
def moving_average(x, w):
    y_padded = np.pad(x, (w//2, w-1-w//2), mode='edge')
    y_smooth = np.convolve(y_padded, np.ones((w,))/w, mode='valid')
    return y_smooth

# original waveform

In [ ]:
fig = plt.figure(figsize=(8,4.5))
train_audio = audio_data[220*samplerate:240*samplerate]
t = np.linspace(0,20,len(train_audio))
plt.xlabel('Time ($s$)')
plt.ylabel('Amplitude ($V$)')
plt.xlim([0,20])
plt.ylim([-0.015,0.015])
plt.plot(t,train_audio,color='#0098D4',lw=0.5)
plt.savefig('final_figures/features/original_wavform_a',dpi=200)

In [ ]:
fig = plt.figure(figsize=(8,4.5))
train_audio = audio_data[250*samplerate:270*samplerate]
t = np.linspace(0,20,len(train_audio))
plt.xlabel('Time ($s$)')
plt.ylabel('Amplitude ($V$)')
plt.xlim([0,20])
plt.ylim([-0.015,0.015])
plt.plot(t,train_audio,color='#0098D4',lw=0.5)
plt.savefig('final_figures/features/original_wavform_b',dpi=200)

In [ ]:
fig = plt.figure(figsize=(8,4.5))
train_audio = audio_data[265*samplerate:285*samplerate]
t = np.linspace(0,20,len(train_audio))
plt.xlabel('Time ($s$)')
plt.ylabel('Amplitude ($V$)')
plt.xlim([0,20])
plt.ylim([-0.015,0.015])
plt.plot(t,train_audio,color='#0098D4',lw=0.5)
plt.savefig('final_figures/features/original_wavform_c',dpi=200)

# spectrogram/mfcc

In [ ]:
f0, t0, Sxx0 = scipy.signal.spectrogram(audio_data[220*samplerate:240*samplerate], fs=samplerate, window='hamming', nperseg=39, noverlap=0, nfft=1000)
pooled = max_pooling(Sxx0, pool_size=(3,3), pool_overlap=(1,1))
lim = [(50*len(pooled))//500, (200*len(pooled))//500]
pooled = pooled[lim[0]:lim[1]]

f = np.linspace(50,200,len(pooled))
t = np.linspace(0,20,len(pooled[0]))
fig = plt.figure(figsize=(8,4.5))
plt.xlabel('Time ($s$)')
plt.ylabel('Frequency ($Hz$)')
plt.pcolormesh(t, f, pooled, norm=colors.LogNorm(vmin=5e-8))
plt.colorbar()
plt.savefig('final_figures/features/spectrogram_a',dpi=200)

In [ ]:
f0, t0, Sxx0 = scipy.signal.spectrogram(audio_data[11*samplerate:31*samplerate], fs=samplerate, window='hamming', nperseg=39, noverlap=0, nfft=1000)
pooled = max_pooling(Sxx0, pool_size=(3,3), pool_overlap=(1,1))
lim = [(50*len(pooled))//500, (200*len(pooled))//500]
pooled = pooled[lim[0]:lim[1]]

f = np.linspace(50,200,len(pooled))
t = np.linspace(0,20,len(pooled[0]))
fig = plt.figure(figsize=(8,4.5))
plt.xlabel('Time ($s$)')
plt.ylabel('Frequency ($Hz$)')
plt.pcolormesh(t, f, pooled, norm=colors.LogNorm(vmin=3e-8))
plt.colorbar()
plt.savefig('final_figures/features/spectrogram_b',dpi=200)

In [ ]:
mfcc = librosa.feature.mfcc(y=audio_data[440*samplerate:460*samplerate],sr=1000,n_mfcc=100, n_mels=10, hop_length=101, fmin=0, fmax=None, htk=False)

In [ ]:
a = [[1,3e-8,2e10],[1,1,1]]
print(np.log(a))
print(np.array(a).flatten())

In [ ]:
fig, ax = plt.subplots(figsize=(8,4.5))
img = librosa.display.specshow(mfcc, x_axis='time', ax=ax)
fig.colorbar(img, ax=ax)
plt.show()

In [ ]:
print(len(mfcc[0]))

# PSD

In [ ]:
fig = plt.figure(figsize=(8,4.5))
f, Pxx = scipy.signal.welch(audio_data[220*samplerate:240*samplerate],fs=3200)
plt.plot(f,Pxx,color='#0098D4', lw=2, label='Train')
f, Pxx = scipy.signal.welch(audio_data[250*samplerate:270*samplerate],fs=3200)
plt.plot(f,Pxx,color='#E32017', lw=2, label='No Train')
plt.xlabel('Frequency ($Hz$)')
plt.ylabel('Power Spectral Density ($dB$)')
plt.xlim(0,1600)
plt.legend()

In [ ]:
fig = plt.figure(figsize=(8,4.5))
a, f = plt.psd(audio_data[220*samplerate:240*samplerate], Fs=3200, return_line=0, scale_by_freq=0,color='#0098D4', lw=2, label='Train')
a, f = plt.psd(audio_data[250*samplerate:270*samplerate], Fs=3200, return_line=0, scale_by_freq=0,color='#E32017',lw=2, label='No Train')
plt.xlabel('Frequency ($Hz$)')
plt.ylabel('Power Spectral Density ($[dB]$ ref $(ms^{-2})^2Hz^{-1}$)')
plt.xlim(0,1600)
plt.legend()
plt.savefig('final_figures/features/psd',dpi=200)
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,4.5))
a, f = plt.psd(audio_data[220*samplerate:240*samplerate], Fs=3200, return_line=0, scale_by_freq=0,color='#0098D4', lw=2, label='Train')
plt.xlabel('Frequency ($Hz$)')
plt.ylabel('Power Spectral Density ($[dB]$ ref $(ms^{-2})^2Hz^{-1}$)')
plt.xlim(0,1600)
plt.savefig('final_figures/features/psd2',dpi=200)
plt.show()

# zero crossing rate
# energy
# max frequency band

In [ ]:
zcr = zero_crossing_rate(audio_data[:samplerate*500])
no_train_zcr = zero_crossing_rate(no_train_audio[:samplerate*500])
maxf = prominent_frequency(audio_data[:samplerate*500], frequency_step=50)
no_train_maxf = prominent_frequency(no_train_audio[:samplerate*500], frequency_step=50)

In [ ]:
fig = plt.figure(figsize=(8,4.5))
train_audio = moving_average(zcr,w=20)
t = np.linspace(0,500,len(train_audio))
plt.plot(t,train_audio,color='#0098D4',lw=1, label='Trains Passing')
train_audio = moving_average(no_train_zcr,w=20)
t = np.linspace(0,500,len(train_audio))
plt.plot(t,train_audio,color='#E32017',lw=1, label='No Passing Train')

plt.xlabel('Time ($s$)')
plt.ylabel('Zero Crossing Rate ($s^{-1}$)')
plt.xlim([0,500])
#plt.ylim([400,600])
plt.legend()
plt.savefig('final_figures/features/zcr',dpi=200)

In [ ]:
energy = short_time_energy(audio_data[:samplerate*500])
no_energy = short_time_energy(no_train_audio[samplerate*500:samplerate*1000])

In [ ]:
energy = moving_average([np.sqrt(x) for x in energy],10)
no_energy = moving_average([np.sqrt(x) for x in no_energy],10)
fig = plt.figure(figsize=(8,4.5))
train_audio = energy
t = np.linspace(0,500,len(train_audio))
plt.xlabel('Time ($s$)')
plt.ylabel('RMS ($V$)')
plt.xlim([0,500])
plt.plot(t,train_audio,color='#0098D4',lw=1, label='Trains Passing')
plt.plot(t,moving_average(no_energy,10),color='#E32017',lw=1,label='No Passing Trains')
plt.legend()
plt.savefig('final_figures/features/rms',dpi=200)
print(len(energy))

In [ ]:
test, samplerate = librosa.load('train_recordings/hour0101.wav', sr=3200)
energy1 = short_time_energy(test[samplerate*700:samplerate*900])

In [ ]:
#energy1 = [np.sqrt(x) for x in energy1]
fig = plt.figure(figsize=(8,4.5))
train_audio = energy1
t = np.linspace(0,200,len(train_audio))
plt.xlabel('Time ($s$)')
plt.ylabel('RMS ($V$)')
plt.xlim([0,200])
plt.ylim([0,1.1813776649012748e-05])
plt.plot(t,moving_average(train_audio,25),color='#0098D4',lw=2)
plt.plot(t,[(1.1813776649012748e-5/np.sqrt(2)) for i in t],color='#E32017',lw=1)
plt.axvline(102,color='#E32017',lw=1)
plt.axvline(116.5,color='#E32017',lw=1)
plt.axhline(1e-6,xmin=0.512,xmax=0.583,color='#E32017',lw=1)
plt.annotate('Half-Power',xy=(0.1,0.7),xycoords='axes fraction',xytext=(0.2, 0.95), textcoords='axes fraction',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"),
            horizontalalignment='right', verticalalignment='top')

plt.annotate('Pulse Duration',xy=(0.55,0.08),xycoords='axes fraction',xytext=(0.8, 0.5), textcoords='axes fraction',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"),
            horizontalalignment='right', verticalalignment='top')

plt.savefig('final_figures/features/pulse',dpi=200)
print(max(moving_average(train_audio,25)))

In [ ]:
energy125 = short_time_energy(audio_data, octave_band=125)
energy63 = short_time_energy(audio_data, octave_band=63)

In [ ]:
print(len(energy125))

In [ ]:
fig = plt.figure(figsize=(8,4.5))
energy125 = [np.sqrt(x) for x in energy125]
train_audio = energy125[440:480]
t = np.linspace(0,20,len(train_audio))
plt.plot(t,moving_average(train_audio,3),color='#0098D4',lw=2, label='Train')
train_audio = energy125[500:540]
t = np.linspace(0,20,len(train_audio))
plt.plot(t,moving_average(train_audio,3),color='#E32017',lw=2, label='No Train')
plt.xlabel('Time ($s$)')
plt.ylabel('$125$Hz $1/{3^{rd}}$ Octave Energy ($V^2$)')
plt.xlim([0,20])
plt.legend()
plt.savefig('final_figures/features/rms2',dpi=200)
print(len(energy))

In [ ]:
fig = plt.figure(figsize=(8,4.5))
energy63 = [np.sqrt(x) for x in energy63]
train_audio = energy63[440:480]
t = np.linspace(0,20,len(train_audio))
plt.plot(t,moving_average(train_audio,4),color='#0098D4',lw=2, label='Train')
train_audio = energy63[500:540]
t = np.linspace(0,20,len(train_audio))
plt.plot(t,moving_average(train_audio,4),color='#E32017',lw=2, label='No Train')
plt.xlabel('Time ($s$)')
plt.ylabel('$63$Hz $1/{3^{rd}}$ Octave Energy ($V^2$)')
plt.xlim([0,20])
plt.legend()
plt.savefig('final_figures/features/rms3',dpi=200)
print(len(energy))

# Peak Value

In [ ]:
peak = peak_value(audio_data, frame_length=1600, hop_length=800)

In [ ]:
fig = plt.figure(figsize=(8,4.5))
train_audio = peak[450:490]
t = np.linspace(0,20,len(train_audio))
plt.plot(t,moving_average(train_audio,7),color='#0098D4',lw=2, label='Train')
train_audio = peak[500:540]
t = np.linspace(0,20,len(train_audio))
plt.plot(t,moving_average(train_audio,7),color='#E32017',lw=2, label='No Train')
plt.xlabel('Time ($s$)')
plt.ylabel('Peak Value ($V$)')
plt.xlim([0,20])
plt.legend()
plt.savefig('final_figures/features/peak',dpi=200)
print(len(peak))

# Spectral Centroid

In [ ]:
speccen = spectral_centroid(audio_data,n_fft=3200 ,fft_hop_length=3200)[0]

In [ ]:
print(len(speccen))

In [ ]:
fig = plt.figure(figsize=(8,4.5))
train_audio = speccen[0:500]
t = np.linspace(0,500,len(train_audio))
plt.xlabel('Time ($s$)')
plt.ylabel('Spectral Centroid ($Hz$)')
plt.xlim([0,500])
plt.plot(t,moving_average(train_audio,5),color='#0098D4',lw=1)
a = [90,157,200,300,360,499]
for i in a:
    plt.axvline(i,color='#E32017',lw=2)
plt.axvline(90,color='#E32017',lw=2,label='Expected Train')
plt.legend()
plt.savefig('final_figures/features/speccen',dpi=200)
print(len(train_audio))

# Spectral Roll-off

In [ ]:
specroll = spectral_roll_off(audio_data, n_fft=3200, fft_hop_length=3200)[0]
print(len(specroll))

In [ ]:
fig = plt.figure(figsize=(8,4.5))
train_audio = specroll[0:500]
t = np.linspace(0,500,len(train_audio))
plt.xlabel('Time ($s$)')
plt.ylabel('Spectral Roll-off ($Hz$)')
plt.xlim([0,500])
plt.plot(t,moving_average(train_audio,5),color='#0098D4',lw=1)
a = [90,157,200,300,360,499]
for i in a:
    plt.axvline(i,color='#E32017',lw=2)
plt.axvline(90,color='#E32017',lw=2,label='Expected Train')
plt.legend()
plt.savefig('final_figures/features/specroll',dpi=200)
print(len(train_audio))

In [ ]:
plt.plot(np.linspace(0,500,len(maxf[0:500])),maxf[0:500])
plt.show()

In [ ]:
trains_array = find_train_trainbar_array_datetime('judged_timetables/judged_windows_cen_eas.csv')

In [ ]:
print(len(trains_array[13]))

In [ ]:
hours=['00','01','02','03','04','05','06','07','08','09','10','11','12','13']
output = rolling_power_with_labels_02(hours, trains_array, octave_bands=[125])

In [ ]:
print(len(output))

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

In [ ]:
gm = GaussianMixture(n_components=2, random_state=0).fit(output[:8000])

In [ ]:
gm.means_

In [ ]:
pred = gm.predict(output[8000:])
print(pred[100:200])
print([i[1] for i in output[8100:8200]])

In [ ]:
print ("gmm: silhouttte: ", silhouette_score(output[8000:], pred))

In [ ]:
regr = svm.SVR()
x_train = np.array([i[0] for i in output[:8000]]).reshape(-1,1)
y_train = [i[1] for i in output[:8000]]
regr.fit(x_train, y_train)

In [ ]:
x_test = np.array([i[0] for i in output[8000:]]).reshape(-1,1)
y_test = np.array([float(i[1]) for i in output[8000:]]).reshape(-1,1)
y_pred = regr.predict(x_test)

In [ ]:
from sklearn import metrics
y_pred = [1 if x>0.5 else 0 for x in y_pred]
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# naive bayes

In [ ]:
gnb = GaussianNB()
y_pred = gnb.fit(x_train, y_train)
print(gnb.score(x_test,y_test))


In [ ]:
print("Number of mislabeled points out of a total %d points : %d" %(x_test.shape[0], (y_test != y_pred).sum()))